In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
import os
sys.path.append(os.path.abspath('../'))
from api import API
from data import Data
import pandas as pd

## Closest Matching UG Playlist to an ED playlist

In [3]:
d = Data()

In [23]:
diffs, ugPIDs = d.closestMatchingUGPlaylist("Are & Be")

In [25]:
ugPIDs[0]

213590

In [26]:
diffs[0]

1.466596422333839

In [27]:
names, artists = d.getUGPlaylist(213590)

In [28]:
names

['Tiny Little Robots',
 'Shoot to Thrill',
 'Immigrant Song',
 'Welcome to Paradise',
 'Hysteria',
 'Supermassive Black Hole',
 'Uprising',
 "The Kids Aren't Alright",
 'Bad Habit',
 'Coming for You',
 'Tom Sawyer',
 'Brianstorm',
 'N.I.B.',
 'Children of the Grave - Remastered Version',
 "I'm Shipping Up To Boston",
 'The State Of Massachusetts',
 'Paradise City',
 'Welcome To The Jungle',
 'For Whom The Bell Tolls - Remastered',
 'A Moment of Violence',
 'Everything Went Numb',
 'Icky Thump',
 'Ace of Spades',
 'Come Out And Play (Keep ‘Em Separated)',
 'Enter Sandman',
 'The Trooper - 1998 Remastered Version',
 'Hallowed Be Thy Name - 1998 Remastered Version',
 'Holy Wars...The Punishment Due - 2004 Digital Remaster',
 'Black Dragon',
 'Collapse (Post-Amerika)',
 'Woman']

In [29]:
artists

['Cage The Elephant',
 'AC/DC',
 'Led Zeppelin',
 'Green Day',
 'Muse',
 'Muse',
 'Muse',
 'The Offspring',
 'The Offspring',
 'The Offspring',
 'Rush',
 'Arctic Monkeys',
 'Black Sabbath',
 'Black Sabbath',
 'Dropkick Murphys',
 'Dropkick Murphys',
 "Guns N' Roses",
 "Guns N' Roses",
 'Metallica',
 'Streetlight Manifesto',
 'Streetlight Manifesto',
 'The White Stripes',
 'Motörhead',
 'The Offspring',
 'Metallica',
 'Iron Maiden',
 'Iron Maiden',
 'Megadeth',
 'The Vines',
 'Rise Against',
 'Wolfmother']

## Predicting playlist avg features from playlist names

In [4]:
features, names = d.getUGPlaylistFeaturesAndNames()

In [5]:
len(features)

1000000

In [6]:
features[:50]

[[0.7822692307692307,
  0.6592115384615383,
  5.038461538461538,
  -4.8784807692307695,
  0.6923076923076923,
  0.10718846153846154,
  0.08298072884615385,
  0.00067601,
  0.19113846153846153,
  0.6426153846153846,
  123.00753846153846,
  4.0],
 [0.690128205128205,
  0.4947153846153846,
  4.461538461538462,
  -8.288435897435898,
  0.5384615384615384,
  0.08908461538461539,
  0.16404996025641028,
  0.2244309776923077,
  0.18570769230769232,
  0.4754871794871795,
  124.90761538461538,
  3.769230769230769],
 [0.693203125,
  0.671875,
  5.0,
  -4.87415625,
  0.515625,
  0.0962875,
  0.26922984375,
  0.0006378143749999999,
  0.169028125,
  0.565078125,
  114.600671875,
  4.0],
 [0.6212420634920635,
  0.513563492063492,
  5.103174603174603,
  -9.614833333333333,
  0.7142857142857143,
  0.0672015873015873,
  0.27384541428571424,
  0.20205686214285715,
  0.18883412698412697,
  0.45190079365079366,
  125.03249206349206,
  3.9523809523809526],
 [0.6505352941176471,
  0.576764705882353,
  3.35294

In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import Normalizer

C:\Users\Adam Khazi\AppData\Local\conda\conda\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
namesPD = pd.DataFrame(names, columns=['text'])

In [9]:
scaler = Normalizer()
scaledFeatures = scaler.fit_transform(features)

In [10]:
scaledFeatures[0].shape

(12,)

In [11]:
namesPD['text'] = namesPD['text'].apply(lambda x: x.lower())
namesPD['text'] = namesPD['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [12]:
max_features = 200
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(namesPD['text'].values)
X = tokenizer.texts_to_sequences(namesPD['text'].values)
X = pad_sequences(X)

In [13]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(12, activation='linear'))
model.compile(loss = 'mean_squared_error', optimizer='adam', metrics = ['mse'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 6, 128)            25600     
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 6, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 12)                2364      
Total params: 282,764
Trainable params: 282,764
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
scaledFeatures.shape

(1000000, 12)

In [16]:
X.shape

(1000000, 6)

In [17]:
Y = scaledFeatures
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(670000, 6) (670000, 12)
(330000, 6) (330000, 12)


In [18]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
 - 262s - loss: 8.4631e-05 - acc: 0.9999
Epoch 2/7
 - 248s - loss: 4.3082e-05 - acc: 1.0000
Epoch 3/7
 - 254s - loss: 4.2246e-05 - acc: 1.0000
Epoch 4/7
 - 253s - loss: 4.1852e-05 - acc: 1.0000
Epoch 5/7
 - 250s - loss: 4.1624e-05 - acc: 1.0000
Epoch 6/7
 - 255s - loss: 4.1530e-05 - acc: 1.0000
Epoch 7/7
 - 264s - loss: 4.1460e-05 - acc: 1.0000


In [19]:
validation_size = 15000

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.00
acc: 1.00
